In [22]:
from collections import defaultdict
import pandas as pd
from surprise import Reader, Dataset
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

import matplotlib.pyplot as plt
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
class RefinedMyAlgo():
    def __init__(self, rating_data='', data_frame='', movie_data=''):
        if rating_data:
            reader = Reader(line_format='user item rating timestamp', sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
#             self.trainset, self.testset = train_test_split(self.ratings, test_size=0.25)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'movieId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            
        if movie_data:
            self.movies = pd.read_csv(movie_data, low_memory=False)
            self.movies['year'] = self.movies['title'].apply(lambda x: x[-5:-1])
            self.movies['title'] = self.movies['title'].apply(lambda x: x[:-7])
            self.movies['genres'] = self.movies['genres'].apply(lambda x: x.replace('|',', '))

        
    def set_k(self, k_value=''):
        if k_value:
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            self.algo = algo
            self.algo.fit(self.trainset)
        else:
            algo = SVD()
            self.algo = algo
            self.algo.fit(self.trainset)
        
        
    def find_best_k(self, k_value=''):
        if k_value:
            print('K = {}'.format(k_value))
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            return cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=True)
        else:
            aux = []
            for k_value in [3, 5, 7, 10, 15, 20, 30, 40]:
                print('K = {}'.format(k_value))
                algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
                my_dict = cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=False)
                my_dict['k_value'] = k_value
                aux.append(my_dict)
            return aux
    
    
    def set_testset(self, users):
        if users:
            user_ratings = self.trainset.ur
            movies_ids = list(self.movies['movieId'])
            global_mean=self.trainset.global_mean
            my_testset = []
            
            for user in users:
                iuid = self.trainset.to_inner_uid(str(user))
                for movie in movies_ids:
                    is_in = False
                    for rating in user_ratings[iuid]:
#                         print( 'MOVIE: {}, RATING: {}'.format(movie,bla.trainset.to_raw_iid(rating[0])) )
                        if int(movie) == int(self.trainset.to_raw_iid(int(rating[0]))):
                            is_in = True
                            break
                    if not is_in:
                        my_tuple = (str(user),str(movie),global_mean)
                        my_testset.append(my_tuple)
                        
            self.testset = my_testset
        else:
            testset = self.trainset.build_anti_testset()
            self.testset = testset
        return self.testset


    def predict_ratings(self,users=''):
        # # Predict ratings for all pairs (u, i) that are NOT in the training set.
#         testset = self.trainset.build_anti_testset()
#         self.testset = testset
        testset = self.set_testset(users)
        predictions = self.algo.test(testset)
        self.predictions = predictions
        
        
    def set_perfil_movies(self, users):
        metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        metadata = metadata.drop(columns="timestamp")

        metadata_filtered = metadata[metadata.userId.isin(users)]

        self.group_sparse_mtx = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
        
        self.perfil_movies = list(self.group_sparse_mtx)
        
    
    ### You must call self.set_perfil_movies() before
    def set_candidate_movies(self):
        candidate_movies = []
        for item in refinedMyAlgo.movies.iterrows():
        #     get the movieId of each movie in movies dataframe
            if item[1].values[0] not in self.perfil_movies:
                candidate_movies.append(item[1].values[0])
        self.candidate_movies = candidate_movies
        
        
    def calc_similarity_matrix(self):
        #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
        tfidf = TfidfVectorizer(stop_words='english')
        
        #Replace NaN with an empty string
        self.movies['title'] = self.movies['title'].fillna('')
        self.movies['genres'] = self.movies['genres'].fillna('')
        
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix_title = tfidf.fit_transform(self.movies['title'])
        tfidf_matrix_genres = tfidf.fit_transform(self.movies['genres'])
        
        #Compute the cosine similarity matrix
        self.cosine_sim_movies_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)
        self.cosine_sim_movies_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
        
        
    def get_similar_movies(self, references, title_weight=0.5):
        recs = []
        for movie in references:
            # Get the pairwsie similarity scores of all movies with that movie
            movie_idx = int(self.movies[self.movies['movieId']==movie].index[0])
            sim_scores_title = list(enumerate(self.cosine_sim_movies_title[movie_idx]))
            sim_scores_genres = list(enumerate(self.cosine_sim_movies_genres[movie_idx]))
            
            # Calculate total similarity based on title and genres
            total_sim_score = []
            for i in range(len(sim_scores_title)):
                aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
                total_sim_score.append((i, aux))
                
            # Sort the movies based on the similarity scores
            total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
            
            candidates_sim_score = []
            for item in total_sim_score:
                if self.movies.loc[item[0]].values[0] not in self.perfil_movies:
                    candidates_sim_score.append(item)
            
            # Get the scores of the 10 most similar movies
            candidates_sim_score = candidates_sim_score[1:11]
            
            recs.append(candidates_sim_score)
            
        return recs

In [24]:
refinedMyAlgo = RefinedMyAlgo(rating_data='ml-latest-small/ratings.csv', movie_data='ml-latest-small/movies.csv')
refinedMyAlgo.set_k()

In [25]:
my_users = [77,596,452,243,420]

refinedMyAlgo.predict_ratings(users=my_users)
len(refinedMyAlgo.predictions)

47891

In [26]:
# metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
# metadata = metadata.drop(columns="timestamp")

# metadata_filtered = metadata[metadata.userId.isin(my_users)]

# my_group_sparse = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
# my_group_sparse.head()

refinedMyAlgo.set_perfil_movies(users=my_users)
refinedMyAlgo.set_candidate_movies()

# print(refinedMyAlgo.perfil_movies)
# print(refinedMyAlgo.candidate_movies)
refinedMyAlgo.group_sparse_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,0,0,0.0,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
243,0,5,0.0,0,4,0,4,0,4,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
420,4,0,3.5,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
452,0,4,0.0,0,0,0,4,5,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
596,4,0,3.5,4,0,4,0,0,0,3.5,...,2.5,3.5,3.5,4,3.5,3.5,3.5,3.5,4,4


In [27]:
group_filled_mtx = refinedMyAlgo.group_sparse_mtx.copy()

for index, row in group_filled_mtx.iterrows():
    for col in list(group_filled_mtx):
        if(group_filled_mtx.loc[index,col] == 0.0):
            aux = list(filter(lambda x: x.uid==str(index) and x.iid==str(col), refinedMyAlgo.predictions))
            group_filled_mtx.loc[index,col] = aux[0].est

group_filled_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,4.169579,3.419081,4.125197,3.775521,3.879171,3.479497,2.738448,3.884791,3.558569,4.048887,...,3.388120,3.355151,3.467183,3.365014,3.139702,3.341831,3.446866,3.666160,3.635402,3.505275
243,4.470238,5.000000,4.543697,4.296309,4.000000,4.215512,4.000000,4.406173,4.000000,4.794921,...,4.159435,3.981909,4.124891,4.317722,3.718445,4.456090,4.021945,4.183446,4.165917,4.184039
420,4.000000,3.513102,3.500000,3.853479,3.930641,3.631898,2.838406,3.880008,3.382756,4.278852,...,3.375915,3.498666,3.513842,3.617806,3.338759,3.625256,3.530445,3.574235,3.710398,3.491090
452,4.927577,4.000000,4.858591,4.418769,4.768687,4.232333,4.000000,5.000000,4.170289,5.000000,...,4.318025,4.233607,4.322167,4.420688,4.305347,4.586347,4.556196,4.606573,4.650078,4.378197
596,4.000000,3.419916,3.500000,4.000000,3.501081,4.000000,2.690210,3.848470,3.185245,3.500000,...,2.500000,3.500000,3.500000,4.000000,3.500000,3.500000,3.500000,3.500000,4.000000,4.000000


In [28]:
########################################################################
# # Implementing least misery ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(min(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,4,3.41908,3.5,3.77552,3.50108,3.4795,2.69021,3.84847,3.18525,3.5,...,2.5,3.35515,3.46718,3.36501,3.1397,3.34183,3.44687,3.5,3.6354,3.49109


In [29]:
group_pref_dict = []
for col in list(agg_group_perf):
    my_dict = {}
#     print('Valor: {}, Coluna: {}'.format(agg_group_perf.loc[900,col], col))
    my_dict['rating'] = agg_group_perf.loc[900,col]
    my_dict['movieID'] = col
    group_pref_dict.append(my_dict)
    
group_pref_dict = sorted(group_pref_dict, key = lambda i: i['rating'],reverse=True)
group_pref_dict

[{'rating': 4.257742892324572, 'movieID': 904},
 {'rating': 4.256467573689837, 'movieID': 38061},
 {'rating': 4.2078048278748845, 'movieID': 57669},
 {'rating': 4.1682972847771715, 'movieID': 6350},
 {'rating': 4.121681482580645, 'movieID': 168252},
 {'rating': 4.068312341282494, 'movieID': 76093},
 {'rating': 4.06131046938519, 'movieID': 318},
 {'rating': 4.058553287426928, 'movieID': 122916},
 {'rating': 4.030577549473217, 'movieID': 1089},
 {'rating': 4.028291322914255, 'movieID': 5618},
 {'rating': 4.015722168928417, 'movieID': 2160},
 {'rating': 4.010465918243902, 'movieID': 899},
 {'rating': 4.0045676568735304, 'movieID': 48394},
 {'rating': 4.002553852291927, 'movieID': 1222},
 {'rating': 4.001419862216525, 'movieID': 4011},
 {'rating': 4.0, 'movieID': 1},
 {'rating': 4.0, 'movieID': 260},
 {'rating': 4.0, 'movieID': 364},
 {'rating': 4.0, 'movieID': 593},
 {'rating': 4.0, 'movieID': 750},
 {'rating': 4.0, 'movieID': 1028},
 {'rating': 4.0, 'movieID': 1136},
 {'rating': 4.0, 'mo

In [30]:
refinedMyAlgo.calc_similarity_matrix()

In [31]:
references=[33649, 57669, 43376]
recs = refinedMyAlgo.get_similar_movies(references)

In [32]:
count = 0
for reference in references:
    print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference].values[0][1],
                                              refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference].values[0][2]))
    for movie in recs[count]:
        
        movie_id = refinedMyAlgo.movies.loc[movie[0]].values[0]
        movie_title = refinedMyAlgo.movies.loc[movie[0]].values[1]
        movie_genres = refinedMyAlgo.movies.loc[movie[0]].values[2]
        print('\tScore: {},\tmovieId: {},\ttitle: {},\tgenres: {}'.format(movie[1], movie_id, movie_title, movie_genres))
        
    count=count+1

Referência: Saving Face	 gêneros: Comedy, Drama, Romance
	Score: 0.6143249334852072,	movieId: 156,	title: Blue in the Face,	genres: Comedy, Drama
	Score: 0.5578633128156909,	movieId: 107141,	title: Saving Mr. Banks,	genres: Comedy, Drama
	Score: 0.5283440968834665,	movieId: 491,	title: Man Without a Face, The,	genres: Drama
	Score: 0.5059725740736212,	movieId: 3831,	title: Saving Grace,	genres: Comedy
	Score: 0.5,	movieId: 4,	title: Waiting to Exhale,	genres: Comedy, Drama, Romance
	Score: 0.5,	movieId: 11,	title: American President, The,	genres: Comedy, Drama, Romance
	Score: 0.5,	movieId: 52,	title: Mighty Aphrodite,	genres: Comedy, Drama, Romance
	Score: 0.5,	movieId: 58,	title: Postman, The (Postino, Il),	genres: Comedy, Drama, Romance
	Score: 0.5,	movieId: 94,	title: Beautiful Girls,	genres: Comedy, Drama, Romance
	Score: 0.5,	movieId: 195,	title: Something to Talk About,	genres: Comedy, Drama, Romance
Referência: In Bruges	 gêneros: Comedy, Crime, Drama, Thriller
	Score: 0.500000

In [33]:
recs[0]

[(129, 0.6143249334852072),
 (8317, 0.5578633128156909),
 (428, 0.5283440968834665),
 (2863, 0.5059725740736212),
 (3, 0.5),
 (10, 0.5),
 (47, 0.5),
 (52, 0.5),
 (83, 0.5),
 (165, 0.5)]

In [21]:
# references=[33649]
# title_weight=0.5

# recs = []
# for movie in references:
#     # Get the pairwsie similarity scores of all movies with that movie
#     movie_idx = int(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==movie].index[0])
#     sim_scores_title = list(enumerate(refinedMyAlgo.cosine_sim_movies_title[movie_idx]))
#     sim_scores_genres = list(enumerate(refinedMyAlgo.cosine_sim_movies_genres[movie_idx]))

#     # Calculate total similarity based on title and genres
#     total_sim_score = []
#     for i in range(len(sim_scores_title)):
#         aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
#         total_sim_score.append((i, aux))

#     # Sort the movies based on the similarity scores
#     total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
    
#     candidates_sim_score = []
#     for item in total_sim_score:
#         if refinedMyAlgo.movies.loc[item[0]].values[0] not in refinedMyAlgo.perfil_movies:
#             candidates_sim_score.append(item)

#     # Get the scores of the 10 most similar movies
#     candidates_sim_score = candidates_sim_score[1:11]
    
#     print('CANDIDATES SIMILARITIES: {}'.format(candidates_sim_score))

#     recs.append(total_sim_score)
    